In [ ]:
import sys,os,re
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd
import seaborn           as sb

from theory_code.distance_theory import TheoryCalcs

from itertools import product
from copy      import deepcopy
from time      import time

from scipy.interpolate import interp1d
from scipy.integrate   import trapz

import warnings
warnings.filterwarnings('ignore')

import matplotlib
from matplotlib import rc
from matplotlib.pyplot import cm
from matplotlib.colors import LogNorm

rc('text', usetex=True)
rc('font', family='serif')
matplotlib.rcParams.update({'font.size': 18})


sidelegend = {'bbox_to_anchor': (1.04,0.5), 
              'loc': "center left",
              'frameon': False}


In [ ]:
fiducial = {'H0': 68.53,
            'omegam': 0.2948,
            'ombh2': 0.02218,
            'omk': 0.,
            'epsilon0_EM': 0.1,
            'epsilon0_GW': -0.1,
            'mnu': 0.06,
            'nnu': 3.,
            'MB': -19.2435}
settings = {'zmin': 0.001,
            'zmax': 5.,
            'Nz': 1000,
            'zdrag': 1060,
            'DDR_model': 'constant'}



In [ ]:
#theory
theory = TheoryCalcs(settings,fiducial)
z_camb  = np.linspace(0.001,5.,10000)
dA      = theory.dA(z_camb)
Hz      = theory.Hz(z_camb)*3*1e5
dL_EM_theo = theory.DL_EM(z_camb)
dL_GW_theo = theory.DL_GW(z_camb)

dL_0       = (1+z_camb)**(2)*dA


In [ ]:
bin_edges = np.linspace(0.65, 1.85, 14)
z_bins    = (bin_edges[:-1] + bin_edges[1:]) / 2
DH        =  theory.DH(z_bins)*theory.rdrag
DV        = theory.DV(z_bins)*theory.rdrag
DM        = theory.DM(z_bins) *theory.rdrag
DH_error = 0.05 * DH
DV_error = 0.05 * DV
DM_error = 0.05 * DM

DH_noisy = np.random.normal(DH, DH_error)
DV_noisy = np.random.normal(DV, DV_error)
DM_noisy = np.random.normal(DM, DM_error)


In [ ]:
Hz_data = 2.99 * 10**5 / DH
dA_data =  (DV**3 / (z_bins * (1 + z_bins)**2 * DH))**0.5
Hz_error = 0.05*Hz_data
dA_error = 0.05*dA_data
Hz_shift = np.random.normal(Hz_data, Hz_error)
dA_shift = np.random.normal(dA_data, dA_error)

plt.figure(figsize=(8, 6))
plt.plot(z_camb, dA, label="Theoretical dA", color='red', linewidth=2)
plt.errorbar(z_bins, dA_shift, yerr=dA_error, fmt='o',label='dA mock')
plt.xlim(0.3, 2.0)
plt.ylim(1300, 2100)
plt.xlabel("Redshift (z)")
plt.ylabel("DA")
plt.title("DA vs Redshift")
plt.legend()
plt.show()

In [ ]:

plt.figure(figsize=(8, 6))
plt.plot(z_camb, Hz, label="Theoretical Hz", color='red', linewidth=2)
plt.errorbar(z_bins, Hz_shift, yerr=Hz_error, fmt='o',label='Hz mock')
plt.xlim(0.3, 2.0)
plt.ylim(50, 250)
plt.xlabel("Redshift (z)")
plt.ylabel("Hz")
plt.title("Hz vs Redshift")
plt.legend()
plt.show()

In [ ]:
#SN
N_SN = 8800

def dN_dz(z):
    return 1.53e-4 * ((1 + z) / 1.5) ** 2.14 * (fiducial['H0']/70)**3


z         = np.linspace(0.1,1.0, 10000)
p_z       = dN_dz(z)
p_z      /= np.sum(p_z) 
z_sampled = np.random.choice(z, size=N_SN, p=p_z, replace=False)

z_SN      = np.sort(z_sampled)



dL_EM      = theory.DL_EM(z_SN)
mock_mu   = interp1d(z_SN, 5 * np.log10(dL_EM) + 25)

mu        = mock_mu(z_SN)



In [ ]:
mu_theoretical = 5 * np.log10(dL_EM_theo) + 25


In [ ]:
sigma_mu = mu*0.005

sigma_dL_EM = 0.05 * dL_EM 
mu_noisy = np.random.normal(mu, sigma_mu)
z_plot= np.linspace(0.1,1.0,10000)


plt.figure(figsize=(8, 6))
plt.errorbar(z_SN, mu_noisy, fmt='o', yerr=sigma_mu, label="Distance Modulus vs Redshift", 
             capsize=2, markersize=5, alpha=0.8,  zorder=1)

plt.plot(z_camb, mu_theoretical, label="Theoretical Distance Modulus", color='red', linewidth=3, zorder=2)

plt.xlim(0.1,1.0)
plt.ylim(35,45)
plt.xlabel("Redshift (z)")
plt.ylabel("Distance Modulus (mu)")
plt.title("Distance Modulus vs Redshift")
plt.legend()
plt.show()

In [ ]:

covmat = np.zeros((len(sigma_mu), len(sigma_mu)))
np.fill_diagonal(covmat, sigma_mu ** 2)
data_SN        = {'z': z_SN,
                'mu': mu,
                'covmat': covmat}
np.save('./mock_data/SN_mock_data.npy', data_SN)
del covmat

data_BAO        = {'z' : z_bins,
                             'DH': DH,
                             'err_DH': DH_error,
                             'DV': DV,
                             'err_DV': DV_error,
                             'DM': DM,
                             'err_DM': DM_error}

error_BAO= np.array([data_BAO['err_DH'], data_BAO['err_DV'], data_BAO['err_DM']])
covmat = np.zeros((len(error_BAO), len(error_BAO)))

np.fill_diagonal(covmat, error_BAO ** 2)
data_BAO['covmat'] = covmat

np.save('./mock_data/BAO_mock_data.npy', data_BAO)
del covmat

In [ ]:
N_gw = 1000
inclination = 0  
G = 6.67430e-11  
solar_mass = 1.9885e30  
m1 = m2 = 1.4 * solar_mass
c = 2.99 * 1e8

In [ ]:
def merger_rate(z):
    if z <= 1:
        return 1 + 2 * z
    elif 1 < z < 5:
        return 3 / 4 * (5 - z)
    else:
        return 0

def chirp_mass(m1, m2):
    return (m1 * m2)**(3/5) / (m1 + m2)**(1/5)

def calculate_amplitude(dL_GW,sigma_gw, m1, m2):
    
    m_chirp = chirp_mass(m1, m2)
    amplitude = 4*(G*m_chirp)**(5/3) /(c**4* dL_GW*3.086 *1e22) #frequency? missing a factor (pi*f)^2/3
    sigma_amplitude = amplitude * 0.1
    
    return amplitude, sigma_amplitude

In [ ]:


r_values = np.array([2.99 * 10**5/theory.Hz(z) for z in z_camb])
merger_rates = np.array([merger_rate(z) for z in z_camb])
p_z = 4 * np.pi * r_values**2 * merger_rates / (theory.Hz(z_camb) * (1 + z_camb))
p_z /= np.sum(p_z)  

    
z_sampled = np.random.choice(z_camb, size=N_gw, p=p_z, replace=False)
z_GW = np.sort(z_sampled)

dL_GW = theory.DL_GW(z_GW)

In [ ]:

snr = np.random.uniform(10, 100, size=N_gw)  
sigma_L = 0.05 * z_GW * dL_GW               
sigma_i = 2 * dL_GW / snr                    
sigma_dLgw = np.sqrt(sigma_L**2 + sigma_i**2) 


dL_GW = np.random.normal(dL_GW, sigma_dLgw)

amplitude,sigma_amplitude = calculate_amplitude(dL_GW, sigma_dLgw, m1,m2)

covmat = np.zeros((len(sigma_dLgw), len(sigma_dLgw)))
np.fill_diagonal(covmat, sigma_dLgw ** 2)

data_GW = { 'z': z_GW,
            'dL_GW': dL_GW,
            'covmat': covmat}
np.save('./mock_data/GW_mock_data.npy', data_GW)


In [ ]:
plt.figure(figsize=(8, 6))


plt.errorbar(data_GW['z'], data_GW['dL_GW'], yerr=sigma_dLgw, 
             fmt='o', color='orchid',ecolor='gray', capsize=2, markersize=2, alpha=0.8,  zorder=1, label='Observed dL (GW)')

plt.plot(z_camb, dL_GW_theo, label="Theoretical dL (GW)", color='red', linewidth=3, zorder=2)
plt.plot(z_camb, dL_0, label="Theoretical dL with $\epsilon=0$", color='green', linewidth=3, zorder=3, ls= '--')

plt.xlabel('Redshift (z)')
plt.ylabel('Luminosity Distance $d_L$ (Mpc)')
plt.title('$d_L$  vs. z $\epsilon_{GW}=-0.1$')
plt.legend()
plt.show()


In [ ]:
N = 100
random_indices = np.random.choice(len(z_SN), N, replace=False)

z_SN_subset = z_SN[random_indices]
dL_EM_subset = dL_EM[random_indices]
sigma_subset = sigma_mu[random_indices]

sigma_dL_subset = (np.log(10) / 5) * dL_EM_subset * sigma_subset

dL_EM_shifted =  np.random.normal(dL_EM_subset, sigma_dL_subset)

plt.figure(figsize=(8, 6))
plt.errorbar(z_SN_subset, dL_EM_shifted, yerr=sigma_dL_subset, color='darkslategrey',
             fmt='o', ecolor='green', capsize=2, markersize=2, alpha=0.8,  zorder=1, label='Observed dL (EM)')
plt.plot(z_camb, dL_EM_theo, label="Theoretical dL", color='blue',ls = '--', linewidth=3, zorder=2)
plt.plot(z_camb, dL_0, label="Theoretical dL with $\epsilon=0$", color='red', linewidth=3, zorder=3, ls= '--')
plt.xlim(0.1, 1.0)
plt.ylim(0.1, 10000)
plt.xlabel('Redshift (z)')
plt.ylabel('Luminosity Distance $d_L$ (Mpc)')
plt.title('$d_L$  vs. z $\epsilon_{EM}=0.1$')
plt.legend()
plt.show()


In [ ]:

plt.figure(figsize=(8, 6))


plt.errorbar(data_GW['z'], data_GW['dL_GW'], yerr=sigma_dLgw,
             fmt='o', color='orchid',ecolor='gray', capsize=2, markersize=2, alpha=0.8,  zorder=1, label='Observed dL (GW)')

plt.plot(z_camb, dL_GW_theo, label="Theoretical dL (GW)", color='red', linewidth=3, zorder=3)
plt.errorbar(z_SN_subset, dL_EM_shifted, yerr=sigma_dL_subset, color='darkslategrey',
             fmt='o', ecolor='green', capsize=2, markersize=2, alpha=0.8,  zorder=4, label='Observed dL (EM)')
plt.plot(z_camb, dL_EM_theo, label="Theoretical dL", color='blue',ls = '--', linewidth=3, zorder=2)
plt.xlabel('Redshift (z)')
plt.ylabel('Luminosity Distance $d_L$ (Mpc)')
plt.title('$d_L$  vs. z $\epsilon_{EM}=0.1$, $\epsilon_{GW}=-0.1$')
plt.legend()
plt.show()

In [ ]:

plt.figure(figsize=(8, 6))


plt.errorbar(data_GW['z'], data_GW['dL_GW'], yerr=sigma_dLgw,
             fmt='o',color='orchid', ecolor='gray', capsize=2, markersize=3, alpha=0.8,  zorder=1, label='Observed dL (GW)')

plt.plot(z_camb, dL_GW_theo, label="Theoretical dL (GW)", color='red', linewidth=3, zorder=3)
plt.errorbar(z_SN_subset, dL_EM_shifted, yerr=sigma_dL_subset, color='darkslategrey',
             fmt='o', ecolor='green', capsize=2, markersize=3, alpha=0.8,  zorder=4, label='Observed dL (EM)')
plt.plot(z_camb, dL_EM_theo, label="Theoretical dL", color='blue',ls = '--', linewidth=3, zorder=2)
plt.xlim(0.1, 1.0)

plt.ylim(0.1, 10000)
plt.xlabel('Redshift (z)')
plt.ylabel('Luminosity Distance $d_L$ (Mpc)')
plt.title('$d_L$  vs. z $\epsilon_{EM}=0.1$, $\epsilon_{GW}=-0.1$')
plt.legend()
plt.show()

In [ ]:

m_chirp = chirp_mass(m1, m2)
amplitude_theo = 4*(G*m_chirp)**(5/3) /(c**4* dL_GW_theo*3.086 *1e22)
plt.figure(figsize=(8, 6))

plt.errorbar(data_GW['z'], amplitude, yerr=sigma_amplitude, 
             fmt='o', color='black',ecolor='gray', capsize=4, markersize=2, alpha=0.8,  zorder=2, label='Observed h (GW)')

plt.plot(z_camb, amplitude_theo, label="Theoretical h (GW)", color='red', linewidth=3, zorder=2)

plt.xlabel('Redshift (z)')
plt.ylabel('Strain amplitude')
plt.title('$h$  vs. z $\epsilon_{GW}=-0.1$')
plt.legend()
plt.show()